In [1]:
# this is using the hsnt kernel conda environment

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

from __code.white_beam import WhiteBeam

from __code import system
system.System.select_working_dir()
from __code.__all import custom_style
custom_style.style()

# Select top sample folder (containing all the sample runs)

In [2]:
o_white_beam = WhiteBeam(system=system)
o_white_beam.select_top_sample_folder()

Top sample folder selected: /SNS/VENUS/IPTS-33699/shared/autoreduce/mcp/September20_2024_PurpleCar_GoldenRatio_CT_5_0_C_Cd_inBeam_Resonance


# Select top OB folder (containing all the OB runs)

In [3]:
o_white_beam.select_top_ob_folder()

Top ob folder selected: /SNS/VENUS/IPTS-33699/shared/autoreduce/mcp/September19_2024_OpenBeam_28_0_C_Resonance


# Pre-processing the data

 

## cleaning runs

- looking at proton charge requested
- looking at empty runs
- looking at uncompatible runs (wrong proton charge, missing bins)

In [4]:
o_white_beam.checking_data()

interactive(children=(FloatSlider(value=5.0, description='sample pc', max=50.0, min=0.01), FloatSlider(value=5…

This will recap the list of runs that will be used for SAMPLE and OB. 

**Feel free to select any run you want to exclude !**

In [6]:
o_white_beam.recap_data()

## combine all the images within each run

In [ ]:
o_white_beam.combine_images()

In [10]:
# pre-processing
## remove "dead" pixels
## remove abnormal high counts

In [11]:
# normalization

In [12]:
# chips correction

In [13]:
# calculate center of rotation

In [14]:
# calculate tilt correction and apply it

In [15]:
# launch svmbir